# Explicação do código
Este código foi desenvolvido para processar dois conjuntos de dados: um contendo dados de pesquisas e outro com dados sobre instituições de ensino superior (IES) no Brasil. O objetivo é limpar e selecionar informações relevantes de ambos os conjuntos de dados e, em seguida, combiná-los com base no estado (UF).

Vamos dividir o código seção por seção:

Esta seção importa duas bibliotecas Python necessárias:

`pandas:` Esta é uma biblioteca fundamental para manipulação e análise de dados, fornecendo estruturas de dados como DataFrames. É comumente usada para ler, limpar, transformar e analisar dados tabulares.

`re:` Este é o módulo integrado do Python para expressões regulares, que são usadas para correspondência de padrões em strings. Neste código, ele será usado para limpar nomes de colunas.
# Parte 1: Limpeza e seleção da pesquisa "State of Datda"

In [3]:
import pandas as pd
import re
import numpy as np

In [4]:
# --- Parte 1: Limpeza e Seleção do "State of Data" (usando sua lógica) ---

print("Iniciando Parte 1: Limpeza e seleção do 'State of Data'...")
# Carrega o arquivo CSV. A primeira linha (cabeçalho) contém os nomes das colunas no formato "('código', 'descrição')".
df_survey = pd.read_csv('State_of_data_BR_2023_Kaggle - df_survey_2023.csv')


Iniciando Parte 1: Limpeza e seleção do 'State of Data'...


Esta parte começa imprimindo uma mensagem indicando o início do processo. Em seguida, carrega um arquivo CSV `'State_of_data_BR_2023_Kaggle - df_survey_2023.csv'`em um DataFrame do Pandas chamado df_survey. Este DataFrame agora contém os dados brutos da pesquisa.





In [5]:
# --- Lógica de Seleção de Colunas (Original Mantida) ---
fixed_columns = [
    "('P1_a ', 'Idade')", "('P1_b ', 'Genero')", "('P1_c ', 'Cor/raca/etnia')",
    "('P1_l ', 'Nivel de Ensino')", "('P2_f ', 'Cargo Atual')", "('P2_g ', 'Nivel')",
    "('P2_h ', 'Faixa salarial')", "('P2_c ', 'Numero de Funcionarios')", "('P2_b ', 'Setor')",
    "('P2_r ', 'Atualmente qual a sua forma de trabalho?')", "('P2_s ', 'Qual a forma de trabalho ideal para você?')",
    "('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')",
    "('P2_k ', 'Você está satisfeito na sua empresa atual?')", "('P1_i_1 ', 'uf onde mora')"
]

p2_l_columns = [col for col in df_survey.columns if col.startswith("('P2_l_")]
p2_o_columns = [col for col in df_survey.columns if col.startswith("('P2_o_")]
p4_j_columns = [col for col in df_survey.columns if col.startswith("('P4_j_")] # Ferramentas de IA/ML
p4_k_columns = [col for col in df_survey.columns if col.startswith("('P4_k_")] # Serviços de Cloud
p4_l_columns = [col for col in df_survey.columns if col.startswith("('P4_l_")] # Tecnologias de IA Generativa
p3_d_columns = [col for col in df_survey.columns if col.startswith("('P3_d_")] # Habilidades Técnicas
p3_e_columns = [col for col in df_survey.columns if col.startswith("('P3_e_")] # Habilidades Comportamentais

selected_columns = (fixed_columns + p2_l_columns + p2_o_columns +
                    p4_j_columns + p4_k_columns + p4_l_columns +
                    p3_d_columns + p3_e_columns)

selected_columns = [col for col in selected_columns if col in df_survey.columns]
df_selected = df_survey[selected_columns].copy()

# --- Limpeza e Tratamento de Nulos (Original Mantido) ---
target_col = "('P2_k ', 'Você está satisfeito na sua empresa atual?')"
df_selected.loc[:, target_col] = df_selected[target_col].astype(str)
df_cleaned_survey = df_selected.dropna(subset=[target_col])
df_cleaned_survey = df_cleaned_survey[df_cleaned_survey[target_col] != 'Desconhecido']

dynamic_prefixes = ("('P2_l_", "('P2_o_", "('P4_j_", "('P4_k_", "('P4_l_", "('P3_d_", "('P3_e_")

for col in df_cleaned_survey.columns:
    if col.startswith(dynamic_prefixes):
        df_cleaned_survey.loc[:, col] = df_cleaned_survey[col].fillna(0)
    else:
        df_cleaned_survey.loc[:, col] = df_cleaned_survey[col].fillna('Desconhecido')

print("'State of Data' limpo e selecionado com sucesso.")


# --- PRINCIPAL ALTERAÇÃO: Renomeação Inteligente das Colunas ---

# --- PRINCIPAL ALTERAÇÃO: Renomeação Inteligente das Colunas ---

def get_clean_column_name(col_name):
    """
    Extrai a parte descritiva do nome da coluna e a limpa para ser usada como um identificador.
    Exemplo: "('P4_j_1 ', 'Azure Machine Learning')" -> "azure_machine_learning"
    """
    try:
        # Usa regex para encontrar o texto da descrição de forma segura
        match = re.search(r",\s*'(.*)'\)", col_name)
        if match:
            name = match.group(1).strip()
            # Converte para minúsculas e substitui espaços e outros caracteres por underscore
            clean_name = re.sub(r'[\s/,-]+', '_', name.lower())
            # Remove underscores duplicados que possam ter sido criados
            return re.sub(r'__+', '_', clean_name)
    except (IndexError, AttributeError):
        # Se a extração falhar, usa o código da coluna como fallback
        pass

    code_match = re.search(r"\('(.+?)',", col_name)
    if code_match:
      return code_match.group(1).strip().replace(' ', '') # Limpa o código também

    return col_name

# Cria o mapa de renomeação usando a função acima para todos os nomes de coluna
rename_map = {col: get_clean_column_name(col) for col in df_cleaned_survey.columns}

# Renomeia todas as colunas de uma vez
df_cleaned_survey.rename(columns=rename_map, inplace=True)

print("Colunas renomeadas com sucesso usando os nomes exatos.")

# **>>>>>> ADICIONE AS SEGUINTES LINHAS AQUI <<<<<<**
print("\nValores únicos na coluna de satisfação após renomeação:")
print(df_cleaned_survey['você_está_satisfeito_na_sua_empresa_atual?'].value_counts(dropna=False))
# **>>>>>> FIM DAS LINHAS A SEREM ADICIONADAS <<<<<<**


# --- Criação da Coluna 'emprego_status' ---
# O nome da coluna 'Cargo Atual' foi alterado para 'cargo_atual' pela função de limpeza.
df_cleaned_survey['emprego_status'] = np.where(df_cleaned_survey['cargo_atual'] == '14', 'Desempregado', 'Empregado')

'State of Data' limpo e selecionado com sucesso.
Colunas renomeadas com sucesso usando os nomes exatos.

Valores únicos na coluna de satisfação após renomeação:
você_está_satisfeito_na_sua_empresa_atual?
1.0    3420
0.0    1333
nan     540
Name: count, dtype: int64


<ipython-input-5-14f0b9644c4d>:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0.0' '1.0' '1.0' ... '1.0' 'nan' 'nan']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_selected.loc[:, target_col] = df_selected[target_col].astype(str)


Aqui, uma lista chamada` fixed_columnsé `criada. Essa lista contém os nomes de colunas específicas do conjunto de dados da pesquisa que são consideradas essenciais para a análise. Os nomes dessas colunas parecem estar em um formato que inclui um código e uma descrição. O comentário indica que a `'P1_i_1 `'coluna (que representa o estado onde a pessoa reside) é crucial para a posterior combinação desses dados com os dados do IES.

Esta parte identifica colunas adicionais do `df_surveyDataFrame`. Ela utiliza compreensões de lista para encontrar nomes de colunas que começam com` "('P2_l_"e "('P2_o_"`. Essas provavelmente representam motivos de insatisfação ou critérios para a escolha de uma empresa, que são consideradas colunas "dinâmicas". As colunas identificadas são então adicionadas à `fixed_columnslista` para criar uma `selected_columnslista` final.

Antes de selecionar as colunas, este código verifica se todos os nomes de colunas` selected_columnsrealmente` existem no` df_surveyDataFrame` original. Esta é uma boa prática para evitar erros caso o nome de uma coluna esteja incorreto ou ausente. Em seguida, ele cria um novo DataFrame chamado , df_selectedque contém apenas o selected_columnsfrom verificado `df_survey`. O .`copy()`é usado para garantir que quaisquer modificações` df_selectednão afetem o df_survey.`

Esta seção se concentra na preparação da "variável alvo", que é a coluna que indica a satisfação com a empresa atual (` '('P2_k ', 'Você está satisfeito na sua empresa atual?')'`).


* A coluna é selecionada e seu nome é armazenado em `target_col`.
* `df_selected.loc[:, target_col] = df_selected[target_col].astype(str) `converte os dados da coluna de destino para o tipo de dados string. Isso garante um tratamento consistente, independentemente do tipo de dado original.
*` df_cleaned_survey `= df_selected.dropna(subset=[target_col])remove todas as linhas df_selectedonde o valor target_colestá faltando (NaN).
*` df_cleaned_survey = df_cleaned_survey`[df_cleaned_survey[target_col] != 'Desconhecido']filtra ainda mais o DataFrame, removendo linhas cujo valor da coluna de destino é "Desconhecido". Isso resulta em um DataFrame df_cleaned_surveyque contém apenas linhas com valores de satisfação válidos e não ausentes.

Este loop itera por todas as colunas no df_cleaned_surveyDataFrame.

Se o nome de uma coluna começar com "('P2_l_"ou "('P2_o_", os valores ausentes ( NaN) nessa coluna serão preenchidos com o número 0. Isso sugere que essas colunas podem representar contagens ou indicadores em que um valor ausente implica uma contagem zero.
Para todas as outras colunas, os valores ausentes são preenchidos com a string 'Desconhecido'. Esta é uma maneira de lidar com dados categóricos ausentes ou outros dados não numéricos. Por fim, uma mensagem é impressa indicando a conclusão do processo de limpeza e seleção dos dados da pesquisa.

# Etapa extra: limpeza de nomes de colunas

Esta seção define uma função clean_col_codeque usa expressões regulares ( re) para extrair a parte do código dos nomes das colunas (a parte entre aspas simples, por exemplo, 'P1_a '). Em seguida, ela aplica essa função a todos os nomes de colunas no df_cleaned_surveyDataFrame, substituindo os nomes originais, mais descritivos, apenas pelos códigos. Isso provavelmente é feito para simplificar os nomes das colunas e facilitar a manipulação e a mesclagem.

Após limpar os nomes das colunas para apenas os códigos, esta parte cria um dicionário survey_rename_mappara mapear esses códigos para nomes mais descritivos e legíveis em inglês. Em seguida, usa o .rename()método para renomear as colunas df_cleaned_surveyde acordo com esse mapeamento.

# Parte 2: Agregação de dados do IES por estado (UF)

In [6]:

# --- Parte 2: Agregação dos Dados das IES por Estado (UF) ---
print("\nIniciando Parte 2: Agregação dos dados das IES...")
try:
    df_ies = pd.read_csv('MICRODADOS_ED_SUP_IES_2023.CSV', delimiter=';', encoding='latin1')
except FileNotFoundError:
    print("Arquivo 'MICRODADOS_ED_SUP_IES_2023.CSV' não encontrado. Pulando esta etapa.")
    df_ies_aggregated = pd.DataFrame(columns=['uf_ies', 'Total_Tecnicos_Estado', 'Total_Docentes_Estado', 'Total_IES_no_Estado']).set_index('uf_ies')
except Exception as e:
    print(f"Um erro ocorreu ao ler o arquivo IES: {e}")
    df_ies_aggregated = pd.DataFrame(columns=['uf_ies', 'Total_Tecnicos_Estado', 'Total_Docentes_Estado', 'Total_IES_no_Estado']).set_index('uf_ies')
else:
    ies_columns_map = { 'SG_UF_IES': 'uf_ies', 'QT_TEC_TOTAL': 'Qtd_Tecnicos_IES', 'QT_DOC_TOTAL': 'Qtd_Docentes_IES' }
    df_ies_selected = df_ies[list(ies_columns_map.keys())].rename(columns=ies_columns_map)

    numeric_cols = ['Qtd_Tecnicos_IES', 'Qtd_Docentes_IES']
    for col in numeric_cols:
        df_ies_selected[col] = pd.to_numeric(df_ies_selected[col], errors='coerce')

    df_ies_aggregated = df_ies_selected.groupby('uf_ies').agg({
        'Qtd_Tecnicos_IES': 'sum',
        'Qtd_Docentes_IES': 'sum',
        'uf_ies': 'count'
    }).rename(columns={'uf_ies': 'Total_IES_no_Estado', 'Qtd_Tecnicos_IES': 'Total_Tecnicos_Estado', 'Qtd_Docentes_IES': 'Total_Docentes_Estado'})
    print("Dados das IES agregados com sucesso.")


Iniciando Parte 2: Agregação dos dados das IES...
Dados das IES agregados com sucesso.


Esta seção começa imprimindo uma mensagem para a segunda parte do processo. Em seguida, lê outro arquivo CSV chamado 'MICRODADOS_ED_SUP_IES_2023.CSV'. Este arquivo é lido com um ponto e vírgula ( ;) como delimitador e 'latin1'como codificação, conforme especificado pelos argumentos de pd.read_csv(). Os dados são carregados em um DataFrame chamado df_ies.

Um dicionário ies_columns_mapé criado para mapear os nomes das colunas originais do conjunto de dados IES ( 'SG_UF_IES', 'QT_TEC_TOTAL', 'QT_DOC_TOTAL') para nomes mais descritivos ( 'uf_ies', 'Qtd_Tecnicos_IES', 'Qtd_Docentes_IES'). Em seguida, ele seleciona essas colunas do df_iesDataFrame e as renomeia usando o mapa fornecido, armazenando o resultado em df_ies_selected.

Este loop itera pelo valor especificado numeric_cols. Para cada coluna, ele tenta converter os dados para um tipo numérico usando pd.to_numeric(). O errors='coerce'argumento significa que, se algum valor não puder ser convertido em um número, ele será substituído por NaN(Não é um número).



Esta é uma etapa fundamental em que os dados do IES são agregados por estado ( 'uf_ies').

df_ies_selected.groupby('uf_ies')agrupa o DataFrame pelos valores na 'uf_ies'coluna.
.agg({...})então aplica funções de agregação aos dados agrupados:
Ele calcula o sumde 'Qtd_Tecnicos_IES'para cada estado.
Ele calcula o sumde 'Qtd_Docentes_IES'para cada estado.
Ele usa 'count'a 'uf_ies'coluna para contar o número de entradas IES dentro de cada estado.
.rename(...)renomeia as colunas agregadas resultantes para que sejam mais descritivas: 'uf_ies'torna-se 'Total_IES_no_Estado', 'Qtd_Tecnicos_IES'torna-se 'Total_Tecnicos_Estado'e 'Qtd_Docentes_Estado'torna-se 'Total_Docentes_Estado'. Os dados agregados são armazenados em df_ies_aggregated. Uma mensagem de sucesso é impressa.

# Parte 3: União final do banco de dados (mesclagem)

In [7]:
# --- Parte 3: União (Merge) Final das Bases de Dados ---
print("\nIniciando Parte 3: União final dos dados...")
# O nome da coluna 'uf onde mora' agora é 'uf_onde_mora'
df_final = pd.merge(
    df_cleaned_survey,
    df_ies_aggregated,
    left_on='uf_onde_mora',
    right_on='uf_ies',
    how='left'
)
print("União finalizada.")


Iniciando Parte 3: União final dos dados...
União finalizada.


Esta parte combina os dados limpos da pesquisa ( df_cleaned_survey) com os dados agregados do IES ( df_ies_aggregated).

pd.merge()é usado para executar a operação de junção.
df_cleaned_surveyé o DataFrame esquerdo.
df_ies_aggregatedé o DataFrame correto.
left_on='uf_residencia'especifica a coluna a df_cleaned_surveyser usada para mesclagem.
right_on='uf_ies'especifica a coluna a df_ies_aggregatedser usada para mesclagem.
how='left'indica uma mesclagem à esquerda. Isso significa que todas as linhas do df_cleaned_survey(DataFrame esquerdo) serão incluídas no resultado. Se um estado de df_cleaned_surveynão tiver uma entrada correspondente em df_ies_aggregated, as colunas de df_ies_aggregatedterão valores ausentes (NaN) para essa linha. O resultado da mesclagem é armazenado em df_final, e uma
mensagem de conclusão é impressa.

# Parte 4: Salvando o Resultado

In [8]:
# --- Parte 4: Salvando o Resultado ---
final_filename = 'dados_tratados_combinados.csv'
df_final.to_csv(final_filename, index=False, encoding='utf-8-sig')

print(f"\nProcesso concluído! Arquivo '{final_filename}' gerado com sucesso.")
print("\nVisualização de algumas colunas do arquivo final, incluindo nomes ajustados:")

# Filtra para colunas que existem antes de tentar imprimir
colunas_para_verificar = [
    'idade', 'genero', 'cargo_atual', 'emprego_status', 'uf_onde_mora', 'total_ies_no_estado'
]
colunas_existentes = [col for col in colunas_para_verificar if col in df_final.columns]
print(df_final[colunas_existentes].head())


Processo concluído! Arquivo 'dados_tratados_combinados.csv' gerado com sucesso.

Visualização de algumas colunas do arquivo final, incluindo nomes ajustados:
   idade     genero                        cargo_atual emprego_status  \
0     31  Masculino  Cientista de Dados/Data Scientist      Empregado   
1     30  Masculino          Analista de BI/BI Analyst      Empregado   
2     37   Feminino     Analista de Dados/Data Analyst      Empregado   
3     22  Masculino                       Desconhecido      Empregado   
4     34  Masculino     Analista de Dados/Data Analyst      Empregado   

  uf_onde_mora  
0           MG  
1           ES  
2           SP  
3           SP  
4           MG  


Esta seção final salva o DataFrame mesclado resultante ( df_final) em um arquivo CSV.

final_filename = 'dados_finais_combinados.csv'define o nome do arquivo de saída.
df_final.to_csv(final_filename, index=False, encoding='utf-8-sig')salva o DataFrame no arquivo CSV especificado.
index=Falseimpede a gravação do índice DataFrame como uma coluna no CSV.
encoding='utf-8-sig'especifica a codificação de caracteres para o arquivo de saída. Por fim, mensagens de sucesso são impressas, incluindo o nome do arquivo gerado, e as 5 primeiras linhas de colunas específicas do df_finalDataFrame são impressas no console, permitindo uma rápida visualização dos dados combinados.